Reviewer 1 asks whether we can really say that, for locking case especially, the change in extreme fraction ($\phi$) is really larger than the change in the precipitation ($P$). 

This is a bit tricky, which is why I didn't do it before. 

We can bootstrap a confidence interval. I guess that we can directly bootstrap the CI for the difference.

$X = \delta \phi - \delta P$

where $\delta() = ( ()_2 - ()_1) / ()_1$

To get at this, we will calculate a very large number of realizations of $X$

In [1]:
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
from collections import namedtuple

Result = namedtuple('Result', ['case', 'gavg', 'goavg', 'tavg', 'toavg', 'toavg_uw', 'p95', 'tsum', 'xsum'])

In [2]:
# locking results:
variable = "PRECT"

tropics = slice(-30,30)

cases = {
    'f-lock': '/Volumes/Samsung_T5/F1850JJB_c201_CLOCK.cam.h2.ncrcat.PRECT.nc',
    'f-cntl': '/Volumes/Samsung_T5/F1850JJB_c201_CTL.cam.h2.ncrcat.PRECT.nc',
    'c-lock': '/Volumes/Samsung_T5/B1850_c201_CLOCK/daily/B1850_c201_CLOCK.cam.h2.ncrcat.PRECT.nc',
    'c-cntl': '/Volumes/Samsung_T5/B1850_c201_CTL/daily/B1850_c201_CTL.cam.h2.ncrcat.PRECT.nc'}

case_names = {'f-lock': 'F1850JJB_c201_CLOCK',
    'f-cntl': 'F1850JJB_c201_CTL',
    'c-lock': 'B1850_c201_CLOCK',
    'c-cntl': 'B1850_c201_CTL'}

# f-cntl ntime = 9126
# f-lock ntime = 9126
# c-cntl ntime = 10982
# c-lock ntime = 9491


# LAND
land_file = xr.open_dataset("/Users/brianpm/Dropbox/Data/cesm2_f09_land.nc")
LAND = land_file['LANDFRAC'].squeeze()


# First things first, let's just load the data:
locking_data = {}
for c in cases:
    tmp = xr.open_dataset(cases[c])
    locking_data[c] = tmp['PRECT'].where(LAND <= 0).sel(lat=tropics)


In [9]:
total_number = locking_data['c-cntl'].values.size
valid_number = np.count_nonzero(~np.isnan(locking_data['c-cntl'].values))  # counts the values that aren't nan
print(f'There are {valid_number} of data out of possible {total_number} elements')

# select the valid values in the flattened array:
flt = locking_data['c-cntl'].values.flatten()
z = flt[~np.isnan(flt)]
# 143 million points -- let's take a sample of 100million w/replacement

There are 143402956 of data out of possible 202420224 elements
(143402956,)


So we have the tropical ocean data for each case in locking_data. 

I suppose what we want to do is take only the tropical ocean data (i.e. non-missing points), and take a sample of that data. Calculate all the pieces, i.e. the total precipitation

In [52]:
rng = np.random.default_rng() # instantiate random number generator

bootstrap_n = 1000
bootstrap_sample_size = np.count_nonzero(~np.isnan(locking_data['c-cntl'].isel(time=0).values))*365*10  # i.e., 10 years of data

result = {}
for c in cases:
    print(f"case is {c}")
    pbar = np.zeros(bootstrap_n)
    phi = np.zeros(bootstrap_n)
    q95 = np.zeros(bootstrap_n)
    diff = np.zeros(bootstrap_n)
    # create the 1-d dataset
    z = locking_data[c].values.flatten()
    z = z[~np.isnan(z)]
    for i in range(bootstrap_n):
        if i%100 == 0:
            print(f"\t ... {i}")
        smpl = rng.choice(z, bootstrap_sample_size, True)
        pbar[i] = np.mean(smpl)
        q95[i] = np.quantile(smpl, 0.95)
        phi[i] = np.nansum(smpl[smpl>q95[i]])/np.nansum(smpl)
    result[c] = (pbar, phi)
    

case is f-lock
	 ... 0
	 ... 100
	 ... 200
	 ... 300
	 ... 400
	 ... 500
	 ... 600
	 ... 700
	 ... 800
	 ... 900
case is f-cntl
	 ... 0
	 ... 100
	 ... 200
	 ... 300
	 ... 400
	 ... 500
	 ... 600
	 ... 700
	 ... 800
	 ... 900
case is c-lock
	 ... 0
	 ... 100
	 ... 200
	 ... 300
	 ... 400
	 ... 500
	 ... 600
	 ... 700
	 ... 800
	 ... 900
case is c-cntl
	 ... 0
	 ... 100
	 ... 200
	 ... 300
	 ... 400
	 ... 500
	 ... 600
	 ... 700
	 ... 800
	 ... 900


In [53]:
# since that took a long time, let's put the results into a file
for c in result:
    d = {"pr_avg": {"dims": ("n"), "data": result[c][0]},
         "phi": {"dims": ("n"), "data": result[c][1]}}
    dstmp = xr.Dataset.from_dict(d)
    fname = f'/Users/brianpm/Dropbox/Manuscripts/M_cre_pex/bootstrap_phi_{c}.nc'
    dstmp.to_netcdf(fname)

In [54]:
# get the quantiles for the CI
for c in result:
    pbar_ci = np.quantile(result[c][0], [0.025, 0.975])
    phi_ci = np.quantile(result[c][1], [0.025, 0.975])
    print(f"For case {c}, 95% CI on Avg(P) is ({pbar_ci[0]},{pbar_ci[1]}), and for phi it is ({phi_ci[0]}, {phi_ci[1]})")

For case f-lock, 95% CI on Avg(P) is (3.963155759123538e-08,3.967842383545417e-08), and for phi it is (0.4095747999846935, 0.40995189025998113)
For case f-cntl, 95% CI on Avg(P) is (4.0505792142653264e-08,4.0558186675809794e-08), and for phi it is (0.432826142758131, 0.43323486149311063)
For case c-lock, 95% CI on Avg(P) is (3.9582340072286115e-08,3.9630223902520355e-08), and for phi it is (0.4172159731388092, 0.4175929225981236)
For case c-cntl, 95% CI on Avg(P) is (4.0317178573445746e-08,4.037019492031391e-08), and for phi it is (0.4287880018353462, 0.4291559934616089)


In [70]:
# next calculate the fractional changes
delta_p = (result['c-lock'][0] - result['c-cntl'][0]) / result['c-cntl'][0]
delta_p_quants = np.quantile(delta_p, [0.025, 0.25, 0.5, 0.75, 0.975])
# print(delta_p_quants)
mean_dp = np.mean(delta_p)
print(f"Mean δP = {mean_dp}, 95% CI = ({delta_p_quants[0]}, {delta_p_quants[-1]})")

delta_phi = (result['c-lock'][1] - result['c-cntl'][1]) / result['c-cntl'][1]
delta_phi_quants = np.quantile(delta_phi, [0.025, 0.25, 0.5, 0.75, 0.975])
# print(delta_phi_quants)
mean_dphi = np.mean(delta_phi)
print(f"Mean δϕ = {mean_dphi}, 95% CI = ({delta_phi_quants[0]}, {delta_phi_quants[-1]})")



diff = delta_phi - delta_p
diff_quants = np.quantile(diff, [0.025, 0.25, 0.5, 0.75, 0.975])
# print(diff_quants)
mean_diff = np.mean(diff)
print(f"Mean δϕ - δP = {mean_diff}, 95% CI = ({diff_quants[0]}, {diff_quants[-1]})")



Mean δP = -0.01829927533448693, 95% CI = (-0.019094793578599827, -0.017391924635068473)
Mean δϕ = -0.026987462001868356, 95% CI = (-0.02761265832381869, -0.02639860001048672)
Mean δϕ - δP = -0.008688186667381423, 95% CI = (-0.009702954683430461, -0.007695971408016631)


In [49]:
def printMinMaxMean(z):
    print(f"Min: {np.min(z):0.5g}, Max: {np.max(z): 0.5g}, Mean: {np.mean(z): 0.5g}")
printMinMaxMean((result['f-cntl'][0])*86400.*1000.)
printMinMaxMean((result['f-lock'][0])*86400.*1000.)

printMinMaxMean((result['f-cntl'][1]))
printMinMaxMean((result['f-lock'][1]))

Min: 3.4817, Max:  3.4896, Mean:  3.4857
Min: 3.4808, Max:  3.49, Mean:  3.4858
Min: 0.42861, Max:  0.42936, Mean:  0.42898
Min: 0.42854, Max:  0.42937, Mean:  0.42898


In [51]:
# Alternative is to use two-sided t-statistic
from scipy import stats

# start from the full data, and let's just see if the means are different
# tropical ocean, removing the land values from the data
case1 = 'f-cntl'
case2 = 'f-lock'
print(f"case 1: {case1}")
v1 = locking_data[case1].values.flatten()
vs1 = v1[~np.isnan(v1)]
print(f"case 2: {case2}")
v2 = locking_data[case2].values.flatten()
vs2 = v2[~np.isnan(v2)]
print("get test stats")
tstat, pval = stats.ttest_ind(vs1,vs2, equal_var=False)
print(f"The t-statistic is {tstat}, the p-value is {pval}; If pvalue is small, the means are different. Check {pval < 0.05}")

case 1: f-cntl
case 2: f-lock
get test stats
The t-statistic is 77.68819340534756, the p-value is 0.0; If pvalue is small, the means are different. Check True


In [50]:
locking_data['f-cntl']

<xarray.DataArray 'PRECT' (time: 9126, lat: 64, lon: 288)>
array([[[8.39906181e-15, 9.56331453e-15, 1.38928625e-14, ...,
         1.08391859e-14, 1.41338747e-14, 1.46511170e-14],
        [9.02741034e-15, 1.01508877e-14, 1.72553571e-14, ...,
         1.06547156e-14, 1.15186333e-14, 1.37354626e-14],
        [1.54575568e-14, 9.32483833e-15, 1.77080589e-14, ...,
         1.10776739e-14, 1.13117777e-14, 1.25204141e-14],
        ...,
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan]],

       [[3.11407833e-09, 2.77238144e-09, 4.07668699e-09, ...,
         1.99464534e-09, 1.90572136e-09, 2.23409780e-09],
        [1.73748882e-09, 2.18656115e-09, 3.34102412e-09, ...,
         1.87943727e-09, 1.97307570e-09, 2.80684742e-09],
        [3.52350771e-09, 3.68159414e-09, 4.37270575e-09, ...,
         1.75055381e-09, 2.52436405e-09, 3.20052651e-09],
...
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan]],

       [[9.52268597e-10, 9.98258143e-10, 5.32153044e-10, ...,
         2.35966602e-09, 4.38641878e-09, 2.02308259e-09],
        [5.71101166e-10, 9.00992725e-10, 3.42386258e-10, ...,
         3.10979154e-09, 2.85298296e-09, 8.87152296e-10],
        [3.15290505e-09, 6.22226215e-10, 7.97934607e-10, ...,
         3.51245810e-09, 1.24410560e-09, 6.31795727e-10],
        ...,
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 -29.69 -28.74 -27.8 -26.86 ... 26.86 27.8 28.74 29.69
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) object 0003-01-01 00:00:00 ... 0028-01-01 00:00:00
Attributes:
    units:         m/s
    long_name:     Total (convective and large-scale) precipitation rate (liq...
    cell_methods:  time: mean

In [ ]:

def get_cesm_pr_info(fil, experiment, land=None):
    tropics = slice(-30,30)
    pr_ds = xr.open_dataset(fil)
    pr = pr_ds['PRECT']*1000. # converts to same units as CMIP6 pr
    lat = pr['lat']
    wgt = np.cos(np.radians(lat))
    gavg = esmlab.weighted_mean(pr.mean(dim=('time','lon')), dim=["lat"], weights=wgt).item()
    tavg = esmlab.weighted_mean(pr.sel(lat=tropics).mean(dim=('time','lon')), dim=["lat"], weights=wgt.sel(lat=tropics)).item()
    if land is not None:
        land_trop = land.sel(lat=tropics)
        msk, _ = xr.broadcast(land, pr)
        pr = pr.where(msk <= 0)
        goavg = esmlab.weighted_mean(pr.mean(dim=('time','lon')), dim=["lat"], weights=wgt).item()
        toavg = esmlab.weighted_mean(pr.sel(lat=tropics).mean(dim=('time','lon')), dim=["lat"], weights=wgt.sel(lat=tropics)).item()
    else:
        goavg = gavg
        toavg = tavg
    ptrop = pr.sel(lat=tropics)
    toavg_uw = ptrop.mean().item()
    p95 = np.nanquantile(ptrop.values.flatten(), 0.95)
    tsum = np.nansum(ptrop.values.flatten())
    xsum = np.nansum(ptrop.where(ptrop>=p95).values.flatten())
    return Result(experiment, gavg, goavg, tavg, toavg, toavg_uw, p95, tsum, xsum)




In [68]:
np.outer(np.ones(10), np.arange(1, 11)) - np.outer(np.arange(1, 11), np.ones(10))

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
       [-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
       [-2., -1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
       [-3., -2., -1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.],
       [-4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.,  5.],
       [-5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.],
       [-6., -5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.],
       [-7., -6., -5., -4., -3., -2., -1.,  0.,  1.,  2.],
       [-8., -7., -6., -5., -4., -3., -2., -1.,  0.,  1.],
       [-9., -8., -7., -6., -5., -4., -3., -2., -1.,  0.]])

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
       [ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.],
       [ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.],
       [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.],
       [ 6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.],
       [ 7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.],
       [ 8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.],
       [ 9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.],
       [10., 10., 10., 10., 10., 10., 10., 10., 10., 10.]])